### Sujet
Vous obtiendrez un ensemble de données, qui montre les notes de films par les utilisateurs.  Téléchargez le jeu de données .  
L'ensemble de données fait partie de l'ensemble de données gratuit Movielens 100-k.  
Dans le fichier Lisez-moi, vous trouverez des informations sur la structure de cet ensemble de données spécifique :

u.data :
- L'ensemble complet de données u, 100 000 évaluations par 943 utilisateurs sur 1 682 éléments.
- Chaque utilisateur a évalué au moins 20 films.
- Les utilisateurs et les éléments sont numérotées consécutivement à partir de 1.
- Les données sont aléatoirement commandé.
- Il s'agit d'une liste séparée par des tabulations
- L'identifiant utilisateur | identifiant de l'article | note | horodatage.
- Les horodatages sont en secondes unix depuis le 01/01/1970 UTC  

### Votre tâche :
- utilisez PySpark pour charger les données et compter la fréquence d'apparition de chaque valeur.  
- Bonuspoints, si vous donnez même le pourcentage d'apparition de chaque valeur.
- Postez une capture d'écran de votre code et de vos résultats.

In [9]:
from pyspark.sql import SparkSession
import pandas as pd

#2- Import StructField, StructType, StringType, IntegerType, FloatType
from pyspark.sql.types import (StructField, IntegerType, StructType)

#8 controle s'il reste des valeurs nulles
from pyspark.sql.functions import desc
import pyspark.sql.functions as F

In [10]:
#Create a spark session and declare it in spark variable

spark = SparkSession.builder.appName('exercice').getOrCreate()

In [11]:
from pyspark.sql.types import StructType, StructField, IntegerType

# user id | item id | rating | timestamp

my_schema = StructType([
    StructField('user_id', IntegerType(), False),
    StructField('item_id', IntegerType(), False),
    StructField('rating', IntegerType(), False),
    StructField('timestamp', IntegerType(), False),
    ]
)

# cf inferSchema = True
df = spark.read.option('delimiter', '\\t').csv('u-data', schema = my_schema)
df.show()

+-------+-------+------+---------+
|user_id|item_id|rating|timestamp|
+-------+-------+------+---------+
|    196|    242|     3|881250949|
|    186|    302|     3|891717742|
|     22|    377|     1|878887116|
|    244|     51|     2|880606923|
|    166|    346|     1|886397596|
|    298|    474|     4|884182806|
|    115|    265|     2|881171488|
|    253|    465|     5|891628467|
|    305|    451|     3|886324817|
|      6|     86|     3|883603013|
|     62|    257|     2|879372434|
|    286|   1014|     5|879781125|
|    200|    222|     5|876042340|
|    210|     40|     3|891035994|
|    224|     29|     3|888104457|
|    303|    785|     3|879485318|
|    122|    387|     5|879270459|
|    194|    274|     2|879539794|
|    291|   1042|     4|874834944|
|    234|   1184|     2|892079237|
+-------+-------+------+---------+
only showing top 20 rows



In [12]:
#groupby pour obtenir le nombre de fils évalués et le pourcentage par user_id

groupby_user_id = df.groupBy("user_id").count()

tot = groupby_user_id.count()

groupby_user_id2 = df.groupBy("user_id") \
  .count() \
  .withColumnRenamed('count', 'Nombre_films_evalues') \
  .withColumn('pourcentage_du_total_évalué', (F.col('Nombre_films_evalues') * 100) / tot ) \
  .sort(desc("pourcentage_du_total_évalué")) \
  .show()

+-------+--------------------+---------------------------+
|user_id|Nombre_films_evalues|pourcentage_du_total_évalué|
+-------+--------------------+---------------------------+
|    405|                 737|          78.15482502651113|
|    655|                 685|          72.64050901378579|
|     13|                 636|          67.44432661717921|
|    450|                 540|          57.26405090137858|
|    276|                 518|          54.93107104984093|
|    416|                 493|         52.279957582184515|
|    537|                 490|          51.96182396606575|
|    303|                 484|          51.32555673382821|
|    234|                 480|          50.90137857900318|
|    393|                 448|          47.50795334040297|
|    181|                 435|         46.129374337221634|
|    279|                 434|          46.02332979851538|
|    429|                 414|          43.90243902439025|
|    846|                 405|          42.9480381760339